# DMFTwDFT


## Introduction
This package will perform DFT+DMFT calculations using free DFT codes (VASP for now). <br>
The file structure for the program is as follows.

![image](tree.png)

## Prerequisites

* gsl
* f2py 
* numpy
* seekpath
* wannier90 (recompile vasp with this)
* Intel compilers (OPTIONAL)  
    -make sure you have the environmental variable **$MKLROOT** set to your mkl library location

## Setting up DMFT



### 1. Compile ksum
   
ksum is the module for achieving DMFT self-consistency. It performs the k-point sum and computes the local Green's function and hybridization function Delta.inp

More details can be found in the following paper (Appendix B) <br>
https://journals.aps.org/prb/pdf/10.1103/PhysRevB.90.235103

 
The following is the **Makefile** used. Set your desired compilers. 

In [ ]:
F77 = mpiifort
PFLAGS = -D_MPI -DMPICH_IGNORE_CXX_SEEK
CFLAGS = $(PFLAGS) -O3 -DAS #-xSSE4.2 not for AMD
FFLAGS = -O2 -xhost

MKLROOT=$MKLROOT
PLIBS=-L$(MKLROOT)/lib/intel64/ \
    -Wl,--start-group \
    -lmkl_intel_lp64 -lmkl_sequential -lmkl_core \
    -Wl,--end-group -lpthread -lm
LIBS = $(PLIBS)

#all : cmp_mu cmp_mu_sp dmft_ksum dmft_ksum_up dmft_ksum_dn
all : dmft_ksum_sp dmft_ksum 

cmp_mu : cmp_mu.f
	$(F77) $(FFLAGS) -o $@ cmp_mu.f $(LIBS)

dmft_ksum_sp : dmft_ksum_sp.f
	$(F77) $(FFLAGS) -o $@ dmft_ksum_sp.f $(LIBS)
dmft_ksum : dmft_ksum.f
	$(F77) $(FFLAGS) -o $@ dmft_ksum.f $(LIBS)

clean :
	rm -f cmp_mu dmft_ksum dmft_ksum_sp 


.SUFFIXES : .cc
.cc.o:
	$(PC++) $(CFLAGS) -c $<

.SUFFIXES : .f
.f.o:
	$(F77) $(FFLAGS) -c $<



Compile it. 

In [ ]:
> make all

This creates the executable **dmft_ksum_sp**. Copy this along with **fort_kpt_tools.f** to the **/bin** directory. 

### 2. Configure Makefile.in

This file is located in the root directory. It will be used to compile both atomd and ctqmc. <br>
Set your desired compilers and GSL_INC,GSL_LIBS depending on your gsl installation.  

In [ ]:
# Please set environment variable WIEN_DMFT_ROOT (in ~/.bashrc) to the same path as DESTDIR in this Makefile
DESTDIR = $(WIEN_DMFT_ROOT)

F90 = ifort 
F77 = ifort
preproc = fpp

WFOPT  = -ansi-alias -O2 -FR -override-limits -prec-div -openmp #-O3 -FR -override-limits -openmp -mp1 -w -prec_div -pc80 -pad -DINTEL_VML -mavx 
FFLAGS = -ansi-alias -O2 -fPIC -prec-div -openmp #-O3 -fPIC -O3 -openmp -mavx -no-prec-div
F2PL =

C++ = icpc # C++ compiler
OFLAGS = -O3 -fopenmp # optimization
GFLAGS = -g -C -fopenmp # debugging

F2PY_LAPACK = --link-lapack_opt

##-------------------- PARALLEL VERSION
#PF90 = /opt/mpich2/intel/11.1/bin/mpif90 #   /opt/mpich2-intel-10.1/bin/mpif90
#WLDFLAGS = -i-static $(FOPT) -L/opt/intel/Compiler/11.1/072/mkl/lib/em64t/ -L/usr/lib64 -pthread 
#WLIBS = -L/opt/intel/Compiler/11.1/072/mkl/lib/em64t/ -lmkl_lapack -lmkl_intel_lp64 -lmkl_intel_thread -lguide -lmkl_core -lpthread # -lmkl_lapack -lmkl -lguide -L/usr/lib64 -pthread

PF90 = mpiifort #mpif90 #   /opt/mpich2-intel-10.1/bin/mpif90
WLDFLAGS = -mkl $(FOPT)  -L/usr/lib64 -pthread 
WLIBS = -mkl

Fmpi_define = -DMPI
CMP = f2py --fcompiler=intelem # fortran to python converter
CMPLIBS = --opt='-fast' --link-lapack_opt
CMPLIBS2 = --f90flags='-openmp ' --opt='-fast' --link-lapack_opt
#PFLAGS = -D_MPI -DMPICH_IGNORE_CXX_SEEK -restrict -ipo -no-prec-div -O2
GSL_INC=/home/uthpala/gsl/include/
GSL_LIBS=/home/uthpala/gsl/lib
PFLAGS = -D_MPI -DMPICH_IGNORE_CXX_SEEK -O3 -I$(GSL_INC) -I$(GSL_INC)/gsl
#
#
# PC++ = /opt/mpich2/intel/11.1/bin/mpicxx -xHOST -O3 -ipo -no-prec-div  # mpicxx # parallel compilers
PC++ = mpicxx
pcc = mpicc # mpicc # parallel compilers
#
#PLIBS = -mkl -L/usr/lib64 -pthread -L/export/home/hyowon/Codes/gsl/lib/ -lgsl -lgslcblas
PLIBS = -mkl -L$(MKLROOT)/lib/intel64 -lmkl_intel_lp64 -lmkl_sequential -lmkl_core -lpthread -L/usr/lib64 -pthread -L$(GSL_LIBS)/ -lgsl -lgslcblas
LLIBS = -L$(MKLROOT)/lib/intel64 -lmkl_intel_lp64 -lmkl_sequential -lmkl_core -lpthread -L/usr/lib64 -pthread


#-------------------- SINGLE PROCESSOR VERSION
#PF90 = ifort
#Fmpi_define = 
#CMP = f2py --fcompiler=intel # fortran to python converter
#WLDFLAGS = $(FOPT) -L/opt/intel/mkl/10.0.011/lib/32 
#WLIBS = -lmkl_lapack -lmkl -lguide -pthread
#
#PC++ = mpicxx # g++ # parallel compilers
#pcc = gcc # parallel compilers
#
#PLIBS = -L/opt/intel/mkl/10.0.1.014/lib/32 -lmkl_lapack -lmkl -lguide -lgsl -pthread
#LLIBS = -L/opt/intel/mkl/10.0.1.014/lib/32 -lmkl_lapack -lmkl -lguide -pthread


### 3.Compile atomd

atomd is the module to compute local atomic interactions for ctqmc.
This part (and the ctqmc code) is written by Kristjan Haule (Rutgers).<br>
**atom_d.py** will generate input files (**info_atom_d.dat**, **actqmc.cix**) for ctqmc run. This is already in the **/bin** directory. 

Compile.

In [ ]:
>make all

Compiling gives the shared objects gaunt.so gutils.so which should be copied into the **/bin** directory.

### 4. Compile ctqmc 

This is the impurity solver used in the code. (Credits:Kristjan Haule)<br>
Compile.

In [ ]:
>make all

Copy the created executable **ctqmc** into the **/bin** directory.

## Running

Now that you have compiled and copied the executables into the **/bin** directory, you are ready to run the program.

Inside the **/runs** directory there is an example for LaNiO3. <br>




Configure **INPUT.py**

In [6]:
################   Input parameters for DFT+DMFT calculations   ##################

###### Main loop parameters ###########

p = {"Niter":     1,               # Number of DFT+DMFT iterations
     "Nit":       5,               # Number of DMFT iterations
     "n_tot":     19,            # Number of total electrons
     "nspin":     1,            # Number of spins
     "atomnames": ['V','O'],       # The name of atoms"],
     "orbs"     : ['d','p'],       # The name  of orbitals
     "L_rot"     : [1,0],       # Whether rotate local axis or not
     "cor_at":    [['V1']],      # Correlated atoms, put degenerate atoms in the same list
     "U":         [5.0],        # Intra-U for each cor_at
     "J":         [1.0],        # Hund's coupling"],
     "Uprime":    [4.8],        # Double counting U'"],
     "cor_orb":   [[['d_z2','d_x2y2'],['d_xz','d_yz','d_xy']]], # DMFT orbitals, other orbitals are treated by HF"],
     "q":         [20,20,20],       # [Nq_x,Nq_y,Nq_z]"],
     "ewin":      [-8,2.9],           # Energy Window
     "nom":       6000,             # Number of Matsubara frequencies"],
     "noms":      1200,            # Number of Matsubara frequencies"],
     "nomlog":    30,             # Number of Matsubara frequencies"],
     "Nforce":    0,             # How many time force will be computed"],
     "dc_type":   1,             # dc type"],
     "mu_iter":   200,                # The chemical potential convergence step"],
     "mix_sig":   0.3,              # Mixing parameter for Sigma"],
     "Nd_qmc":    1,             # DMFT Nd values are obtained from QMC sampling"],
     "print_at":  ['V1'],     # The local Green functions are printed"],
     "path_bin":  "~/Codes/WANDMFT/bin/",     # Path to bin files
     }

##### VASP parameters ########
pV = {"System=":     ["LaNiO3",       "# The name of system"],
      "ENCUT=":      [600.0,                 "# Energy cutoff"], 
      "ISPIN=":      [1,               "#ISPIN"],
      "NBANDS=":     [46,                     "# LMAX"],
      "LMAXMIX=":    [4,             "#LMAX"],
      "ISMEAR=":      [-4,                    "# ISMEAR"],
      "GGA=":    ["PS",             "#GGA"],
      "ADDGRID=":    [".TRUE.",             "#GGA"],
      "LWANNIER90=":      [".TRUE.",                    "# ISMEAR"],
     } 

##### CTQMC parameters #########
pC = {"exe":         ["ctqmc",    "# Path to executable"],
      "Delta":       ["Delta.inp",                 " # Input bath function hybridization"],
      "cix":         ["impurity.cix",              " # Input file with atomic state"],
      "mu":          [0,                           " # Chemical potential"],
      "beta":        [100.0,                       " # Inverse temperature"],
      "M" :          [12000000,                    " # Number of Monte Carlo steps"],
      "nom":         [60,                          " # number of Matsubara frequency points to sample"],
      "Nmax":        [1400,                        " # maximal number of propagators"],
      "Ntau":        [1000,                        " # Ntau"],
      "SampleGtau":  [1000,                        " # Sample Gtau"],
      "sderiv":      [0.005,                       " # maximal discripancy"],
      "CleanUpdate": [50000,                       " # clean update after QMC steps"],
      "aom":         [8,                           " # number of frequency points to determin high frequency tail"],
      "warmup":      [250000,                      "  # Warmup"],
      "GlobalFlip":  [1000000,                      "  # Global flip"],
      "PChangeOrder":[0.9,                         "  # Probability to add/remove interval"],
      "TwoKinks":    [0.,                          "  # Two kinks"],
      "Ncout":       [500000,                      " # Ncout"],
      "Naver":       [80000000,                    "  # Naver"],
      }
############## CIX parameters ###########
pD = {"para="     : 1,
      "l="        : 2,
      "n="        : [6, 7, 8],
      "OCA_G="    : False,
      "CoulombF=" : "Ising",
      }

**OPTIONAL**: Parallel Run<br>
Set **para_com**=mpirun -np < number of processors> in **RUNDMFT.py**.

Edit the VASP input files within INPUT.py as you wish. <br>
It is important to set **LWANNIER90=.TRUE.** in the **INCAR** file. 

Do a regular DFT run to find the Fermi energy. It is in the OUTCAR. Create a file, **DFT_mu.out** and save the fermi energy in it. 

Finally, run the program with the following code. 

In [ ]:
> python RUNDMFT.py

### Instructions to create inputs (INPUT.py)

Let us work with an example, SrVO3.

$V^{+4}$ has one electron in its d-shell. We will treat this electron using DMFT. 

Calculating the number of wannier bands:

$V^{+4}$ electrons (d shell) = 1 <br>
3 $O^{-2}$ electrons (p shell) = 3x6 <br>
total number of electrons = 19 

<br>
To find the energy window, plot the bands of DFT using and determine the energy window of the correlated orbital.